In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from kmeans import K_Means
# from svd import SVD

In [2]:
data = pd.read_excel('data/Course Recommendation System(1-162).xlsx')
data = data.drop(data.iloc[:, 0:6], axis=1) # Delete unnecessary columns
data_core = data.iloc[:, :5]

courses = list(data.columns)
core_courses = list(data_core.columns)
core_courses

['Data Structures and Algorithms',
 'Computer Architecture\n',
 'Discrete Mathematics\n',
 'Economics',
 'Programming-2']

In [3]:
data = data.fillna(0.5)
X = data.to_numpy()
X_core = data_core.to_numpy()
X_electives = X[:, 5:]

X_core contains our user vectors (ratings for each user in the core courses)

In [4]:
# K-means on user vectors

n_clusters = 5
n_init = 10
kmeans = K_Means(n_clusters, n_init)
kmeans.fit(X_core, 100)
kmeans.get_cluster_size()
cluster_assignment = kmeans.c_assignment

In [5]:
# X_comp = np.zeros((1, 25))

# for i in range(0, n_clusters):
#     temp = X_core[np.where(cluster_assignment == i)]
#     mean = np.mean(X_electives[np.where(cluster_assignment == i)], axis=0)
#     mean_repeated = np.repeat(mean.reshape(1, -1), temp.shape[0], axis=0)
#     X_comp = np.vstack((X_comp, np.concatenate((temp, mean_repeated), axis=1)))
    
# X_comp = X_comp[1:, :]

In [120]:
X_comp = np.zeros((1, 25))

for i in range(n_clusters):
    mean = np.mean(X[np.where(cluster_assignment == i)], axis=0).reshape(1, -1)
    X_comp = np.vstack((X_comp, mean))
    
X_comp = X_comp[1:, :]
X_comp.shape

(5, 25)

In [121]:
u, s, vh = np.linalg.svd(X_comp, full_matrices=False)
print(u.shape, s.shape, vh.shape)
np.allclose(X_comp, np.dot(u * s, vh))

(5, 5) (5,) (5, 25)


True

In [123]:
X_svd = np.dot(u * s, vh)

In [155]:
X_svd

array([[0.8375    , 0.68035714, 0.90678571, 0.10535714, 0.83392857,
        0.87142857, 0.83928571, 0.58571429, 0.65357143, 0.63571429,
        0.58571429, 0.37857143, 0.525     , 0.51785714, 0.48214286,
        0.35714286, 0.32142857, 0.30357143, 0.33928571, 0.26785714,
        0.25178571, 0.42857143, 0.36428571, 0.25      , 0.5125    ],
       [0.8525    , 0.465     , 0.185     , 0.09      , 0.87      ,
        0.7625    , 0.59      , 0.5025    , 0.522     , 0.5795    ,
        0.7075    , 0.3875    , 0.3975    , 0.61      , 0.56      ,
        0.28      , 0.22      , 0.21      , 0.23      , 0.185     ,
        0.275     , 0.31      , 0.345     , 0.275     , 0.39      ],
       [0.85803483, 0.82705882, 0.82679932, 0.80430189, 0.86320408,
        0.76621401, 0.75739093, 0.65724386, 0.54747851, 0.55812578,
        0.59852941, 0.53196858, 0.61474387, 0.55882353, 0.59137538,
        0.41647059, 0.42823529, 0.40794118, 0.42573529, 0.42205882,
        0.47352941, 0.47252445, 0.50779412, 0.

In [208]:
def get_predictions(n, X):
	for i in range(X.shape[0]):
		ind = np.argpartition(X[i, :], -n)[-n:]
		ind = ind[np.argsort(X_svd[i, :][ind])]
		print("Cluster - ", i + 1, "\n")
		print(X_svd[i, :][ind][::-1])
		print([courses[i] for i in ind[::-1]], "\n")
		
	
get_predictions(5, X_svd)

Cluster -  1 

[0.90678571 0.87142857 0.83928571 0.8375     0.83392857]
['Discrete Mathematics\n', 'Machine Learning\n', 'Mathematics For Machine Learning\n', 'Data Structures and Algorithms', 'Programming-2'] 

Cluster -  2 

[0.87   0.8525 0.7625 0.7075 0.61  ]
['Programming-2', 'Data Structures and Algorithms', 'Machine Learning\n', 'Software Production Engineering\n', 'Programming Languages\n'] 

Cluster -  3 

[0.86320408 0.85803483 0.82705882 0.82679932 0.80430189]
['Programming-2', 'Data Structures and Algorithms', 'Computer Architecture\n', 'Discrete Mathematics\n', 'Economics'] 

Cluster -  4 

[0.876 0.824 0.756 0.74  0.686]
['Computer Architecture\n', 'Economics', 'Programming-2', 'Machine Learning\n', 'Data Structures and Algorithms'] 

Cluster -  5 

[0.82857143 0.79047619 0.6952381  0.69047619 0.62857143]
['Programming-2', 'Economics', 'Machine Learning\n', 'Data Structures and Algorithms', 'The Web and the Mind '] 



### Manual SVD

In [ ]:
# def QR_Decomposition(A):
#     n, m = A.shape # get the shape of A

#     Q = np.empty((n, n)) # initialize matrix Q
#     u = np.empty((n, n)) # initialize matrix u

#     u[:, 0] = A[:, 0]
#     Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

#     for i in range(1, n):

#         u[:, i] = A[:, i]
#         for j in range(i):
#             u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

#         Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

#     R = np.zeros((n, m))
#     for i in range(n):
#         for j in range(i, m):
#             R[i, j] = A[:, j] @ Q[:, i]

#     return Q, R

# def svd_simultaneous_power_iteration(A, k, epsilon=0.00001):
#     n_orig, m_orig = A.shape
#     if k is None:
#         k=min(n_orig,m_orig)
#     A_orig=A.copy()
#     if n_orig > m_orig:
#         A = A.T @ A
#         n, m = A.shape
#     elif n_orig < m_orig:
#         A = A @ A.T
#         n, m = A.shape
#     else:
#         n,m=n_orig, m_orig
        
#     Q = np.random.rand(n, k)
#     # Q, _ = np.linalg.qr(Q)
#     Q, _ = QR_Decomposition(Q)
#     Q_prev = Q
#     #this part does the block power iteration
#     for i in range(1000):
#         Z = A @ Q
#         # Q, R = np.linalg.qr(Z)
#         Q, R = QR_Decomposition(Z)
#         err = ((Q - Q_prev) ** 2).sum()
#         Q_prev = Q
#         if err < epsilon:
#             break
            
#     singular_values=np.sqrt(np.diag(R))
#     # print(singular_values)
#     # singular_values = singular_values[:-5]
#     # print(singular_values)
#     #deal with different shape input matrices
#     if n_orig < m_orig: 
#         left_vecs=Q.T
#         # left_vecs = left_vecs[:, :-5]
#         print(left_vecs.shape)
#         #use property Values @ V = U.T@A => V=inv(Values)@U.T@A
#         right_vecs=np.linalg.inv(np.diag(singular_values))@left_vecs.T@A_orig
#     elif n_orig==m_orig:
#         left_vecs=Q.T
#         print(left_vecs.shape)
#         right_vecs=left_vecs
#         singular_values=np.square(singular_values)
#     else:
#         right_vecs=Q.T
#         # right_vecs = right_vecs[:-5, :]
#         print(right_vecs.shape)
#         #use property Values @ V = U.T@A => U=A@V@inv(Values)
#         left_vecs=A_orig@ right_vecs.T @np.linalg.inv(np.diag(singular_values))

#     print(left_vecs.shape)
#     print(singular_values.shape)
#     print(right_vecs.shape)
#     return left_vecs, singular_values, right_vecs

In [153]:
import numpy as np

class SVD:
    def __init__(self, A):
        self.u, self.s, self.vh = self.svd(A)

    def gram_schmidt(self, A):
        m, n = A.shape
        Q = np.zeros((m, n))
        R = np.zeros((n, n))

        for j in range(n):
            v = A[:, j]
            for i in range(j):
                R[i, j] = np.dot(Q[:, i], A[:, j])
                v = v - R[i, j] * Q[:, i]
            R[j, j] = np.linalg.norm(v, ord=2)
            Q[:, j] = v / R[j, j]

        return Q, R

    def qr_eigen(self, A, max_iter=1000, tol=1e-6):
        for _ in range(max_iter):
            Q, R = self.gram_schmidt(A)
            A = np.dot(R, Q)
            
            if np.linalg.norm(A - np.diag(np.diag(A)), ord=2) < tol:
                break

        return np.diag(A), Q

    def svd(self, A):
        m, n = A.shape
        if(m < n):
            A = A.T
            m, n = A.shape

        ATA = A.T.dot(A)
        # eigenvals, V = self.eig(ATA)
        eigenvals, V = np.linalg.eig(ATA)


        idx = eigenvals.argsort()[::-1]
        V = V[:, idx]

        singularvals = np.sqrt(np.maximum(eigenvals[idx], 0))
        S = np.zeros((m, n))
        print(singularvals.shape)
        S[:min(m, n), :min(m, n)] = np.diag(singularvals)

        U = np.zeros((m, m))
        for i in range(min(m, n)):
            if singularvals[i] != 0:
                U[:, i] = A.dot(V[:, i]) / singularvals[i]

        if m >= n:
            return U, S, V.T
        else:
            return V, S.T, U.T

    def truncated_svd(self, A, k):
        U, S, Vt = self.svd(A)
		
        Uk = U[:, :k]
        Sk = S[:k, :k]
        Vtk = Vt[:k, :]

        return Uk, Sk, Vtk

    def return_matrices(self):
        return self.u, self.s, self.vh

    def return_trunc_matrices(self, A, k):
        return self.truncated_svd(A, k)


In [154]:
SVD = SVD(X_comp)
u, s, vh = SVD.return_matrices()
print(u.shape, s.shape, vh.shape)
print(np.allclose(X_comp, np.dot(u @ s, vh).T))

(5,)
(25, 25) (25, 5) (5, 5)
True


In [80]:
X_comp

array([[0.8375    , 0.68035714, 0.90678571, 0.10535714, 0.83392857,
        0.87142857, 0.83928571, 0.58571429, 0.65357143, 0.63571429,
        0.58571429, 0.37857143, 0.525     , 0.51785714, 0.48214286,
        0.35714286, 0.32142857, 0.30357143, 0.33928571, 0.26785714,
        0.25178571, 0.42857143, 0.36428571, 0.25      , 0.5125    ],
       [0.8525    , 0.465     , 0.185     , 0.09      , 0.87      ,
        0.7625    , 0.59      , 0.5025    , 0.522     , 0.5795    ,
        0.7075    , 0.3875    , 0.3975    , 0.61      , 0.56      ,
        0.28      , 0.22      , 0.21      , 0.23      , 0.185     ,
        0.275     , 0.31      , 0.345     , 0.275     , 0.39      ],
       [0.85803483, 0.82705882, 0.82679932, 0.80430189, 0.86320408,
        0.76621401, 0.75739093, 0.65724386, 0.54747851, 0.55812578,
        0.59852941, 0.53196858, 0.61474387, 0.55882353, 0.59137538,
        0.41647059, 0.42823529, 0.40794118, 0.42573529, 0.42205882,
        0.47352941, 0.47252445, 0.50779412, 0.

In [77]:
np.dot(u @ s, vh)

array([-11.6177206 ,  -7.27408031, -10.85300483,  -1.89683497,
       -11.46288214, -10.7311457 , -10.2527905 ,  -7.32018666,
        -7.73118324,  -7.61496659,  -7.93594782,  -5.46183959,
        -6.29417681,  -7.590366  ,  -7.10584858,  -4.42860086,
        -3.66766485,  -3.54541712,  -4.07158974,  -3.45926335,
        -3.73895811,  -4.7467616 ,  -4.76373478,  -3.606109  ,
        -5.99747871])

In [65]:
u, s, vh = SVD.return_trunc_matrices(X_comp, 10)
print(u.shape, s.shape, vh.shape)
print(np.allclose(X_comp, np.dot(u @ s, vh)))

TypeError: return_trunc_matrices() missing 1 required positional argument: 'k'

In [66]:
np.dot(u @ s, vh)


array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         3.64285714e-01,  2.50000000e-01,  5.12500000e-01],
       [ 8.00000000e-01,  9.00000000e-01,  9.00000000e-01, ...,
         3.64285714e-01,  2.50000000e-01,  5.12500000e-01],
       [ 1.00000000e+00,  6.00000000e-01,  8.00000000e-01, ...,
         3.64285714e-01,  2.50000000e-01,  5.12500000e-01],
       ...,
       [ 7.00000000e-01,  1.00000000e-01,  8.00000000e-01, ...,
         5.33333333e-01,  5.52380952e-01,  6.28571429e-01],
       [ 1.00000000e+00,  8.58982043e-16, -1.75610868e-15, ...,
         5.33333333e-01,  5.52380952e-01,  6.28571429e-01],
       [ 6.00000000e-01,  3.00000000e-01,  4.00000000e-01, ...,
         5.33333333e-01,  5.52380952e-01,  6.28571429e-01]])